# Introduction to Time Series Data

Data can come in many different formats, and many differentshapes and sizes. You've maybe heard of tabular data, a format you may be familiar with from working in something like Excel. 

We will explore two main kinds of tabular data in this module. The first is time series data. Time series data will be *indexed* with a date and time. We'll look a bit more closely at that soon, but for now just think of it as each row having a date or time, rather than a row number.

## Loading Data

One of the most popular packages in Python for working with tabular data is called Pandas. Today we'll get acquainted with Pandas.

The first thing we'll do is `import` the `pandas` package. Convention has us use a shortform name - `pd` - because we'll be using the package so often.

In [3]:
import pandas as pd

This time, we're not using a pre-packaged dataset - we'll use pandas' `read_csv()` to load Apple trading data from a URL into a `DataFrame`.

In [4]:
# Load the data
url = "https://raw.githubusercontent.com/ImperialCollegeLondon/efds-ta-python/main/data/AAPL_2020.csv"
df = pd.read_csv(url)

We can look at the DataFrame as we usually do.

In [5]:
# Check the head
print(df.head())


         Date       Open       High        Low      Close  Adj Close  \
0  2020-01-02  74.059998  75.150002  73.797501  75.087502  73.347923   
1  2020-01-03  74.287498  75.144997  74.125000  74.357498  72.634850   
2  2020-01-06  73.447502  74.989998  73.187500  74.949997  73.213615   
3  2020-01-07  74.959999  75.224998  74.370003  74.597504  72.869278   
4  2020-01-08  74.290001  76.110001  74.290001  75.797501  74.041489   

      Volume  
0  135480400  
1  146322800  
2  118387200  
3  108872000  
4  132079200  


In [6]:
# Print summary info
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       252 non-null    object 
 1   Open       252 non-null    float64
 2   High       252 non-null    float64
 3   Low        252 non-null    float64
 4   Close      252 non-null    float64
 5   Adj Close  252 non-null    float64
 6   Volume     252 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 13.9+ KB
None


## Setting the Index

In a DataFrame, each row is assigned a unique index value. By default, this is just a number (starting at 0). With our Tips data, we only considered an index when plotting data that we had grouped and aggregated.

When it makes sense, we can choose one of the other columns to be an index. For time series data, where each row represents a different point in time, we'll set our `Date` column as the index. This will make it easier for us to work with the data, and can speed up other operations later on.

First, we have to convert our date column from a plain string to a datetime object.


In [7]:
# Convert the 'Date' column to a datetime object
df['Date'] = pd.to_datetime(df['Date'])

# Set the 'Date' column as the index
df.set_index('Date', inplace=True)

We convert the 'Date' column to a datetime object because pandas can recognise and efficiently work with datetime objects. We set the `Date` column as the index because in time-series data like ours, operations are time-based.

With the index set, we can now use it to access different portions of our data a little bit more easily.

In [8]:
# Access a row
print(df.loc['2020-08-18'])

# Access a specific cell
print(df.loc['2020-08-18', 'Close'])

# Access a range
print(df.loc["2020-08-18":"2020-08-20"])


Open         1.143525e+02
High         1.160000e+02
Low          1.140075e+02
Close        1.155625e+02
Adj Close    1.136640e+02
Volume       1.056336e+08
Name: 2020-08-18 00:00:00, dtype: float64
115.5625
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2020-08-18  114.352501  116.000000  114.007500  115.562500  113.664024   
2020-08-19  115.982498  117.162498  115.610001  115.707497  113.806641   
2020-08-20  115.750000  118.392502  115.732498  118.275002  116.331970   

               Volume  
Date                   
2020-08-18  105633600  
2020-08-19  145538000  
2020-08-20  126907200  


### Exercise 1

Compare AAPL's *median* **high** in Q1 and Q2 of 2020. In which quarter was it higher? Use the cell below to show your work.

In [9]:
## YOUR CODE GOES HERE

## Returns

Returns refer to the gain or loss made on an initial investment, often expressed as a percentage. We can use the generic **percentage change** formula here:

$$ (price_{end} - price_{start}) / price_{start} $$

We can apply this to close prices to calculate the simple daily return:

$$ (close price_{today} - close price_{yesterday}) / close price_{yesterday} $$

When available, use the Adjusted Close price, which takes into account corporate actions (dividends for example).

In [10]:
aug18_closing = df.loc["2020-08-18", "Adj Close"]
aug17_closing = df.loc["2020-08-17", "Adj Close"]

aug18_return = (aug18_closing - aug17_closing) / aug17_closing
print(aug18_return)

0.008332761423410027


This simple daily return expresses a loss in value of 2.5%

If we wanted to use the above approach to calculate daily returns for each day in our data set, it would take a long time. Let's see how we can use pandas `pct_change()` to make this sort of work easy, by applying our percentage change formula one column at a time.

In [11]:
# Create a new column and populate it with daily returns
df['Daily Return'] = df['Adj Close'].pct_change()
df

,Open,High,Low,Close,Adj Close,Volume,Daily Return
Date,,,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,73.347923,135480400,NaN
2020-01-03,74.287498,75.144997,74.125000,74.357498,72.634850,146322800,-0.009722
2020-01-06,73.447502,74.989998,73.187500,74.949997,73.213615,118387200,0.007968
2020-01-07,74.959999,75.224998,74.370003,74.597504,72.869278,108872000,-0.004703
2020-01-08,74.290001,76.110001,74.290001,75.797501,74.041489,132079200,0.016086
...,...,...,...,...,...,...,...
2020-12-23,132.160004,132.429993,130.779999,130.960007,129.030792,88223700,-0.006976
2020-12-24,131.320007,133.460007,131.100006,131.970001,130.025925,54930100,0.007712
2020-12-28,133.990005,137.339996,133.509995,136.690002,134.676407,124486200,0.035766


Notice how the first row in our data has a missing value **NaN** in the new daily return column. This is because our data doesn't have a close price for the day before it!

What to do with this missing value depends on what further analysis we want to do. If we want to carry out simple descriptive statistics like compute the mean, max, or standard deviation,  we can leave our missing value as NaN, because pandas will by default ignore NaNs when calculating these.

In [12]:
print("Average daily return:", df["Daily Return"].mean())

print("Highest daily return:", df["Daily Return"].max())

print("Standard Deviation:", df["Daily Return"].std())

Average daily return: 0.0027690712327727435
Highest daily return: 0.11980822698162852
Standard Deviation: 0.02946918476260677


For more complex analyses though, we may want to drop or fill this value. Let's calculate cumulative returns for the period. Instead of comparing a given day with the day before it, cumulative returns compare a given day with the first day of the period, to indicate how our stock has performed since our initial investment.

Because we're doing cumulative multiplication, we'll add 1 to the adjusted closing price, so we can compound the return over time.

In [ ]:
df["Cumulative Return"] = (1 + df["Daily Return"]).cumprod()

df

,Open,High,Low,Close,Adj Close,Volume,Daily Return,Cumulative Return,20-day MA,200-day MA,20-day SD,colour
Date,,,,,,,,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,73.347923,135480400,0.000000,1.000000,NaN,NaN,NaN,green
2020-01-03,74.287498,75.144997,74.125000,74.357498,72.634850,146322800,-0.009722,0.990278,NaN,NaN,NaN,green
2020-01-06,73.447502,74.989998,73.187500,74.949997,73.213615,118387200,0.007968,0.998169,NaN,NaN,NaN,green
2020-01-07,74.959999,75.224998,74.370003,74.597504,72.869278,108872000,-0.004703,0.993474,NaN,NaN,NaN,red
2020-01-08,74.290001,76.110001,74.290001,75.797501,74.041489,132079200,0.016086,1.009456,NaN,NaN,NaN,green
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-23,132.160004,132.429993,130.779999,130.960007,129.030792,88223700,-0.006976,1.759161,122.277770,97.111182,4.227012,red
2020-12-24,131.320007,133.460007,131.100006,131.970001,130.025925,54930100,0.007712,1.772728,123.063029,97.457493,4.128667,green
2020-12-28,133.990005,137.339996,133.509995,136.690002,134.676407,124486200,0.035766,1.836131,124.053225,97.790657,4.425094,green


### Exercise 2

Calculate the daily change in trading volume for this stock, and then calculate the mean change in trading volume over the period. Take the same approach we used for calculating daily returns, but consider volume instead of the adjusted close price.

Then determine the trend in AAPL's stock this year:

- Rising volume and increasing price might indicate a **bullish** trend (where the uptrend is backed by strong demand and could continue).

- Rising volume and decreasing price might indicate a **bearish** trend (where the downtrend is backed by strong selling pressure and could continue).

- Falling volume on price increase or decrease often indicate a trend is losing strength. It might suggest that momentum is waning and a price reversal is coming.

In [14]:
## YOUR CODE GOES HERE

## Surges

Surges in price or trading volume can be helpful indicators for traders. We usually define a surge as an increase on the day before by an amount higher than some defined threshold. A common threshold is two standard deviations above the mean. Let's look at price surges.

In [15]:
# Find the mean return
mean_return = df["Daily Return"].mean()

# Define a threshold as two standard deviations above the mean
return_threshold = mean_return + (df["Daily Return"].std() * 2)

# Define a condition
condition = df["Daily Return"] > return_threshold

# Subset the dataframe where daily returns are higher than the threshold
df[condition]

,Open,High,Low,Close,Adj Close,Volume,Daily Return,Cumulative Return
Date,,,,,,,,
2020-03-02,70.570000,75.360001,69.430000,74.702499,73.145035,341397200,0.093101,0.997234
2020-03-10,69.285004,71.610001,67.342499,71.334999,69.847748,285290000,0.072022,0.952280
2020-03-13,66.222504,69.980003,63.237499,69.492500,68.043655,370732000,0.119808,0.927683
2020-03-24,59.090000,61.922501,58.575001,61.720001,60.433208,287531200,0.100325,0.823925
2020-04-06,62.724998,65.777496,62.345001,65.617500,64.249451,201820400,0.087237,0.875955
2020-07-31,102.885002,106.415001,100.824997,106.260002,104.326248,374336800,0.104689,1.422348
2020-10-12,120.059998,125.180000,119.279999,124.400002,122.356346,240226800,0.063521,1.668164


## Moving Averages

Moving averages are a different kind of indicator, one that smooths out small variations in trading data to give a better picture of the overall trend.

A Simple Moving Average (SMA) is one which averages out a price over a specific period. The average is "moving" because when a new day is considered in the period, the oldest date is discarded.

Moving averages can be *fast*, when they cover a short period, or *slow* when they consider a longer period. The longer the period, the more those small variations are smoothed out.

In [16]:
# Calculate a fast, 20-Day Moving Average
df['20-day MA'] = df['Adj Close'].rolling(window=20).mean()

# Calculate a slow, 200-Day Moving Average
df['200-day MA'] = df['Adj Close'].rolling(window=200).mean()

## Other useful functions

Another useful package in data science is NumPy. It actually powers alot of the work that pandas does, so technically, you've already used it!

One useful function is `np.where()`, often used for populating columns with a signal or indicator, depending on if a condition is met. Let's create a column to colour code our trading days. Days will have a different colour depening on if the market closes higher (green) or lower (red) than the opening.

In [18]:
import numpy as np

df["colour"] = np.where(df["Close"] > df["Open"], "green", "red")
df

,Open,High,Low,Close,Adj Close,Volume,Daily Return,Cumulative Return,20-day MA,200-day MA,20-day SD,colour
Date,,,,,,,,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,73.347923,135480400,0.000000,1.000000,NaN,NaN,NaN,green
2020-01-03,74.287498,75.144997,74.125000,74.357498,72.634850,146322800,-0.009722,0.990278,NaN,NaN,NaN,green
2020-01-06,73.447502,74.989998,73.187500,74.949997,73.213615,118387200,0.007968,0.998169,NaN,NaN,NaN,green
2020-01-07,74.959999,75.224998,74.370003,74.597504,72.869278,108872000,-0.004703,0.993474,NaN,NaN,NaN,red
2020-01-08,74.290001,76.110001,74.290001,75.797501,74.041489,132079200,0.016086,1.009456,NaN,NaN,NaN,green
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-23,132.160004,132.429993,130.779999,130.960007,129.030792,88223700,-0.006976,1.759161,122.277770,97.111182,4.227012,red
2020-12-24,131.320007,133.460007,131.100006,131.970001,130.025925,54930100,0.007712,1.772728,123.063029,97.457493,4.128667,green
2020-12-28,133.990005,137.339996,133.509995,136.690002,134.676407,124486200,0.035766,1.836131,124.053225,97.790657,4.425094,green
